In [ ]:
import requests
import json

class BambuLabOTA:
    def __init__(self):
        self.account = None
        self.password = None
        self.login_url = "https://bambulab.com/api/sign-in/form"
        self.api_url = "https://api.bambulab.com"
        self.device_id = None
        self.access_token = None
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }
        self.user_account()

    def user_account(self):
        self.account = input("Please enter your Bambu Lab cloud account: ")
        self.password = input("Please enter the password for the Bambu Lab Cloud: ")
        self.login()

    def login(self):
        data = {
            "account": self.account,
            "password": self.password,
            "apiError": "",
        }
        try:
            response = requests.post(self.login_url, headers=self.headers, data=data)
            response.raise_for_status()

            if response.status_code == 200 and response.json().get("tfaKey", "") == "":
                print("BambuLab Cloud Login Successful")
                self.access_token = response.cookies.get("token", None)
                print(f"access token: {self.mask_token(self.access_token)}")
                self.user_bind()
            else:
                print(f"BambuLab Cloud Login Failed Error: {response.json()}")
        except requests.RequestException as e:
            print(f"An error occurred during login: {e}")

    def mask_token(self, token, visible_chars=4):
        return token[:visible_chars] + '*' * (len(token) - visible_chars)

    def user_bind(self):
        headers = {
            **self.headers,
            "authorization": f"Bearer {self.access_token}"
        }
        try:
            response = requests.get(f"{self.api_url}/v1/iot-service/api/user/bind", headers=headers)
            response.raise_for_status()

            if response.status_code == 200:
                devices = response.json().get("devices")
                if devices:
                    device_list_str = ", ".join(f"{index + 1}. {device['dev_id']}" for index, device in enumerate(devices))
                    print(f"List of Binded Devices ({device_list_str}): ")
                    try:
                        selected_index = int(input("Please select a device index: "))
                        selected_device = devices[selected_index - 1]
                        print(f"Selected Device: {selected_device}")
                        self.device_id = selected_device["dev_id"]
                        self.device_version()
                    except (IndexError, ValueError):
                        print("Invalid index selected.")
                else:
                    print("No devices found.")
            else:
                print(f"There was a problem getting bound devices. Error: {response.json()}")
        except requests.RequestException as e:
            print(f"An error occurred while getting bound devices: {e}")

    def device_version(self):
        headers = {
            **self.headers,
            "authorization": f"Bearer {self.access_token}"
        }
        try:
            response = requests.get(
                f"{self.api_url}/v1/iot-service/api/user/device/version?dev_id={self.device_id}",
                headers=headers
            )
            response.raise_for_status()

            if response.status_code == 200:
                print(f"Device Version Information: {response.json()}")
                firmware_optional = {
                    "user_id": "0",
                    "upgrade": {
                        "sequence_id": "0",
                        "command": "upgrade_history",
                        "src_id": 2,
                        "firmware_optional": {
                            "firmware": {
                                "version": "",
                                "url": "",
                                "force_update": False,
                                "description": "",
                                "status": "release"
                            },
                            "ams": [{
                                "dev_model_name": "BL-A001",
                                "address": 0,
                                "device_id": "",
                                "firmware": [{
                                    "version": "00.00.06.40",
                                    "force_update": False,
                                    "url": "https://public-cdn.bambulab.com/upgrade/device/BL-A001/00.00.06.40/product/ams-ota_v00.00.06.40-20230906131441.json.sig",
                                    "description": "",
                                    "status": "testing"
                                }],
                                "firmware_current": None
                            }]
                        }
                    }
                }
                firmware_info = response.json()["devices"][0]["firmware"][0]
                firmware_optional["upgrade"]["firmware_optional"]["firmware"]["version"] = firmware_info["version"]
                firmware_optional["upgrade"]["firmware_optional"]["firmware"]["url"] = firmware_info["url"]
                firmware_optional["upgrade"]["firmware_optional"]["firmware"]["description"] = firmware_info["description"]
                print("\n\n\n=============================================================================================")
                print(json.dumps(firmware_optional, indent=4))
            else:
                print(f"There was a problem getting the device version. Error: {response.json()}")
        except requests.RequestException as e:
            print(f"An error occurred while getting the device version: {e}")

if __name__ == '__main__':
    BambuLabOTA()